In [53]:
# import/install librares/packages
# !pip install statsmodels
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

# initialize scaler
scaler = StandardScaler()

In [7]:
# read in model-ready dataset CSV
df = pd.read_csv("model_ready_dataset.csv")
print(df.columns)
df.head()

Index(['title', 'artist', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'view_count', 'like_count', 'comment_count', 'chart_year',
       'lyrics', 'type_Group', 'type_Person', 'country_CA', 'country_GB',
       'country_Other', 'country_US', 'key_0', 'key_1', 'key_2', 'key_3',
       'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10',
       'key_11', 'duration_min', 'begin_year', 'pop', 'rock', 'hip-hop', 'r&b',
       'country', 'jazz', 'electronic', 'reggae', 'metal', 'folk',
       'lyrics_pos', 'lyrics_neg', 'lyrics_neu', 'lyrics_compound',
       'title_pos', 'title_neg', 'title_neu', 'title_compound',
       'lyrics_subjectivity', 'word_count', 'char_count', 'avg_word_len',
       'unique_words', 'vocab_richness', 'Love_Romantic',
       'Personal_Relationship', 'Self_Introspective', 'Party_Dance',
       'Rap_HipHop', 'Love_Pop', 'Dance_Club', 'Catchy_Hook', 'Spanish_Latin',
  

,title,artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,Love_Romantic,Personal_Relationship,Self_Introspective,Party_Dance,Rap_HipHop,Love_Pop,Dance_Club,Catchy_Hook,Spanish_Latin,Friendship_Group
0,...baby one more time,britney spears,0.20200,0.759,0.699,0.000131,0.4430,-5.745,0.0307,92.960,...,0.000433,0.000433,0.214117,0.000433,0.000433,0.782418,0.000433,0.000433,0.000433,0.000433
1,doo wop (that thing),lauryn hill,0.03930,0.535,0.505,0.000000,0.0923,-8.926,0.2450,99.935,...,0.059881,0.042124,0.000189,0.005295,0.262191,0.392826,0.000189,0.036040,0.002516,0.198749
2,have you ever?,brandy,0.54200,0.698,0.533,0.000000,0.3330,-6.246,0.0437,134.001,...,0.013297,0.347975,0.000327,0.000327,0.000327,0.595981,0.000327,0.040786,0.000327,0.000327
3,love like this,faith evans,0.00364,0.767,0.551,0.000000,0.0451,-7.328,0.0616,100.904,...,0.128983,0.086150,0.000361,0.000361,0.000361,0.764405,0.000361,0.018295,0.000361,0.000361
4,this kiss,faith hill,0.17500,0.398,0.804,0.000000,0.1810,-5.559,0.0451,186.752,...,0.252031,0.095312,0.000483,0.000483,0.000483,0.457520,0.000483,0.052928,0.088748,0.051531


In [51]:
# features
# X = df.drop(columns=['title','artist','like_count','comment_count','view_count', 'lyrics'])
X = df.drop(columns=['title','artist','like_count','comment_count','view_count', 'lyrics', 'lyrics_neg', 'lyrics_pos', 'lyrics_neu', 'title_neg', 'title_pos', 'title_neu', 'char_count', 'Love_Pop', 'type_Person', 'country_US', 'key_0', 'unique_words'])

# numeric columns only
num_cols = X.select_dtypes(include=['float64', 'int64']).columns

# scale numeric columns
X[num_cols] = scaler.fit_transform(X[num_cols])

# target: log-transform view_count
y = np.log1p(df['view_count'])

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# initialize and fit model
mlr = LinearRegression()
mlr.fit(X_train, y_train)

# predict on test set
y_pred_log = mlr.predict(X_test)

# convert predictions back to original scale
y_pred = np.expm1(y_pred_log)  # inverse of log1p
y_true = np.expm1(y_test)

# RMSE on original scale
rmse = root_mean_squared_error(y_true, y_pred)
print("Test RMSE (original scale):", rmse)

# cross-validated RMSE on log scale
neg_mse_scores = cross_val_score(
    mlr, X, y, cv=10, scoring='neg_mean_squared_error'
)
rmse_cv_log = np.sqrt(-neg_mse_scores)
print("Cross-validated RMSE per fold (log scale):", rmse_cv_log)
print("Mean RMSE (log scale):", np.mean(rmse_cv_log))

Test RMSE (original scale): 733843686232.6644
Cross-validated RMSE per fold (log scale): [2.22934032 2.08717232 1.83974678 1.91375618 2.05790361 1.9607256
 2.0849151  2.11856242 1.95804133 2.14281723]
Mean RMSE (log scale): 2.039298089515041


In [56]:
# add constant for statsmodels
X_sm = sm.add_constant(X)

# fit OLS model on log-transformed target
model = sm.OLS(y, X_sm).fit()

# show summary with coefficients, t-stats, p-values
print(model.summary())

# compute VIF for each feature
vif_data = pd.DataFrame()
vif_data['feature'] = X_sm.columns
vif_data['VIF'] = [variance_inflation_factor(X_sm.values, i) for i in range(X_sm.shape[1])]
vif_data = vif_data.sort_values(by='VIF', ascending=False)
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:             view_count   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     24.33
Date:                Sat, 29 Nov 2025   Prob (F-statistic):          4.44e-205
Time:                        23:16:12   Log-Likelihood:                -11585.
No. Observations:                5490   AIC:                         2.328e+04
Df Residuals:                    5437   BIC:                         2.363e+04
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    17.42